# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [36]:
from openai import OpenAI
import os
from dotenv import load_dotenv
import gradio as gr
import json

In [6]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key:
    print(f"API key exists and starts with {openai_api_key[:8]}")
else:
    print("API Key is missing")

API key exists and starts with sk-proj-


In [54]:
open_ai_model = "gpt-4o-mini"
openai = OpenAI(api_key=openai_api_key)
system_prompt = "You're a helpful assistant working for AirBelgium. Your task is to assist users with their inquiries and provide relevant information. You should answer in a concise and friendly manner in one or two sentences. If you don't know the answer, just say you don't know. Do not make up an answer. If you miss some information from the user to perform some action ask them for it."

In [55]:
import base64
from io import BytesIO
from PIL import Image
def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"An image representing a vacation in {city}, showing tourist spots and everything unique about {city}, in a vibrant pop-art style",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))

In [38]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return {"price": ticket_prices.get(city, "Unknown")}

def book_a_flight(destination_city, date):
    print(f"Tool book_a_flight called for {destination_city} on {date}")
    city = destination_city.lower()
    booking_id = f"{city[:3].upper()}{date.replace('-', '')}"
    with open("bookings.txt", "a") as f:
        f.write(f"Flight booked to {city} on {date}\n")
    return {"status": "success", "booking_id": f"{booking_id}"}

def get_my_bookings():
    with open("bookings.txt", "r") as f:
        bookings = f.readlines()
    return {"status": "success", "bookings": [booking.strip() for booking in bookings]}

In [ ]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

book_function = {
    "name": "book_a_flight",
    "description": "Book a flight to the destination city on the specified date. Call this whenever a customer wants to book a flight. for example when a customer says 'I want to book a flight to this city on this date'. If you don't have the date, ask the customer for it.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "date": {
                "type": "string",
                "description": "The date when the customer wants to travel"
            }
        },
        "required": ["destination_city", "date"],
        "additionalProperties": False
    }
}

get_bookings_function = {
    "name": "get_my_bookings",
    "description": "Retrieve the list of all bookings made by the user. Call this whenever a customer wants to see their bookings, for example when a customer says 'Can you show me my bookings?'",
    "parameters": {
        "type": "object",
        "properties": {
        },
        "required": [],
        "additionalProperties": False
    }
}


tools = [{"type": "function", "function": price_function}, {"type": "function", "function": book_function}, {"type": "function", "function": get_bookings_function}]

In [56]:
def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    name = tool_call.function.name
    print(name)
    print(arguments)
    res = globals()[name](**arguments)
    response = {
        "role": "tool",
        "content": json.dumps(res),
        "tool_call_id": tool_call.id
    }
    return response


In [57]:
def chat(history):
    print(history)
    messages = [{"role": "system", "content": system_prompt}] + history
    response = openai.chat.completions.create(model=open_ai_model, messages=messages, tools=tools)
    i = 0
    image = None
    while response.choices[0].finish_reason=="tool_calls" and i < 3:
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        if message.tool_calls[0].function.name == "get_ticket_price":
            #image = artist(json.loads(message.tool_calls[0].function.arguments)["destination_city"])
            pass
        response = openai.chat.completions.create(model=open_ai_model, messages=messages)
        i += 1

    reply = response.choices[0].message.content
    history += [{"role":"assistant", "content":reply}]
    return history, image

In [58]:
def transcribe(voice_input):
    # Use a speech-to-text library or API to transcribe the audio
    with open(voice_input, "rb") as audio_file:
        response = openai.audio.transcriptions.create(
            model="whisper-1",
            file=audio_file
        )
        return response.text
    return ""

In [47]:
def on_submit(message, history, voice_input):
    print(history)
    print(message)
    if voice_input:
        print("VOICE INPUT")
        message = transcribe(voice_input)
        print(message)
    if history is None:
        history = []
    history += [{'role': "user", "content": message}]
    return "", history, None

In [59]:
with gr.Blocks() as ui:
    with gr.Row():
        chat_output = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        input_box = gr.Textbox(label="Chat with our assistant")
        voice_input = gr.Audio(type="filepath", label="Or speak to our assistant")
    with gr.Row():
        submit_btn = gr.Button("Submit")

    submit_btn.click(on_submit, inputs=[input_box, chat_output, voice_input], outputs=[input_box, chat_output, voice_input]).then(
        chat, inputs=chat_output, outputs=[chat_output, image_output]
    )

ui.launch(inbrowser=True)


* Running on local URL:  http://127.0.0.1:7899
* To create a public link, set `share=True` in `launch()`.


[]

VOICE INPUT
Hello, can you book me a flight to Berlin if the price is below $600 please?
[{'role': 'user', 'metadata': None, 'content': 'Hello, can you book me a flight to Berlin if the price is below $600 please?', 'options': None}]
[{'role': 'user', 'metadata': None, 'content': 'Hello, can you book me a flight to Berlin if the price is below $600 please?', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Could you please provide me with the date when you would like to travel to Berlin?', 'options': None}]

VOICE INPUT
Yeah, sure, it would be the 3rd of October 2025.
[{'role': 'user', 'metadata': None, 'content': 'Hello, can you book me a flight to Berlin if the price is below $600 please?', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Could you please provide me with the date when you would like to travel to Berlin?', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'Yeah, sure, it would be the 3rd of October 2025.', 'o